In [1]:
#!nvidia-smi

In [ ]:
#!pip install opencv-python numpy torch torchvision ultralytics matplotlib tqdm timm
#!pip install -q transformers sentencepiece gdown inference-gpu supervision more_itertools scikit-learn umap-learn
#!pip install "inference[transformers,sam,clip,gaze,grounding-dino]"

In [1]:
from utils import read_video, save_video
from tracker import Tracker
from team_assigner import TeamAssigner
from pitch import PitchAnnotator
import torch
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


[10/06/25 08:31:28] WARNING  Your inference package version 0.48.0 is out of date! Please upgrade to ]8;id=81387;file:///home/dragos/coursesDeusto/DeepLearning/project3/.venv/lib/python3.12/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=866833;file:///home/dragos/coursesDeusto/DeepLearning/project3/.venv/lib/python3.12/site-packages/inference/core/__init__.py#41\41]8;;\
                             version 0.58.1 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

cuda


In [2]:
from pitch import SoccerPitchConfiguration
# Dont forget about stubs when using a new video
CONFIG = SoccerPitchConfiguration()

In [3]:
# 1. load all frames
video_frames = read_video('08fd33_4.mp4')

# 2. track players/ref/ball in the original video
tracker = Tracker('models/player_detection.pt')
tracks  = tracker.get_object_tracks(
	video_frames,
	read_from_stub=True,
	stub_path='stubs/track_stubs.pkl'
)
tracker.add_position_to_track(tracks)

# 3. assign each tracked player a team ID/color
team_assigner = TeamAssigner(device='cuda')
team_assigner.assign_teams(
	tracks,
	video_frames,
	read_from_stub=True,
	stub_path='stubs/team_stubs.pkl'
)

Loading crops and player info from stubs/team_stubs.pkl...


Embedding extraction: 17it [00:02,  6.46it/s]
Embedding extraction: 480it [01:02,  7.63it/s]


In [ ]:
# Video with players detected and teams drawn
#-----------------

output_video_frames = tracker.draw_annotations(video_frames, tracks) #team_ball_control

save_video(output_video_frames, 'output_video.avi')

#-----------------

In [3]:
# 4. assign each player a jersey number
pitch_ann = PitchAnnotator(
	api_key  = os.getenv('ROBOFLOW_API_KEY'),
	model_id = 'football-field-detection-f07vi/15',
	vertices = CONFIG.vertices,
	edges    = CONFIG.edges
)

In [ ]:
# Pitch Detection, adds edges and vertices drawn
#---------------

output_frames = []
for frame in video_frames:
	fr = pitch_ann.annotate_frames(frame)
	output_frames.append(fr)

save_video(output_frames, 'pitch_with_edges.avi')

#---------------

In [5]:
# Tactical board pitch conversion
#-----------------

tactical_board = []
for idx, frame in enumerate(video_frames):

	tactical = pitch_ann.annotate_tactical_board(
		frame,                
		tracks,            # positional too, or use keywords from here on
		idx,
		CONFIG,
	)
	tactical_board.append(tactical)

save_video(tactical_board, 'match_with_tactical_board.avi')

#-----------------

In [5]:
# Voronoi pitch detection
#-----------------

voronoi_board = []
for idx, frame in enumerate(video_frames):

	tactical = pitch_ann.annotate_voronoi(
		frame,                
		tracks,            # positional too, or use keywords from here on
		idx,
		CONFIG,
	)
	voronoi_board.append(tactical)

save_video(voronoi_board, 'match_with_voronoi.avi')

#-----------------